In [1]:
import pandas as pd

In [2]:
# name="020529.050002EDT"
# name = name + ".csv"
savename = 'LibBldg1'

prefix = '../data/csv/'
names = ['LibBldg1-{0}'.format(i) for i in range(1, 12)]
suffix = ''

df = pd.concat(pd.read_csv(prefix + name + suffix) for name in names)

In [3]:
len(df)

13193451

In [4]:
df.Protocol.unique()

array(['TCP', 'IPX', 'LLC', 'RIP', 'ARP', 'NBDS', 'UDP', 'BOOTP', 'IPv4',
       'DNS', 'ENIP', 'MDNS', 'ICMP', 'EAPOL', 'NBNS', 'SRVLOC',
       'Elasticsearch', '0x6002', 'IPv6', 'NTP', 'TFTP', 'CN/IP', 'PPPoED',
       'AX4000', 'PKTC', 'SABP', 'POWERLINK/UDP', 'RSVP', 'SEBEK', 'CUPS',
       '0x0000', 'MSproxy', 'TIME', 'RPC', 'ANSI C12.22', 'OpenVPN',
       'HART_IP', 'ASAP', 'BFD Control', 'MANOLITO', 'H.225.0'], dtype=object)

In [8]:
mask = (df["Protocol"]=="TCP") | (df["Protocol"]=="IPv4") | (df["Protocol"]=="UDP")
tcp = df[mask]
tcp, len(tcp)

(             No.          Time          Source      Destination Protocol  \
 0              1      0.000000  214.96.247.160   190.84.116.236      TCP   
 19            20      4.707113   190.84.199.73   190.84.117.150      TCP   
 21            22      4.854644  190.84.117.150    190.84.199.73      TCP   
 22            23      5.049894  190.84.117.150    190.84.199.73      TCP   
 23            24      5.080000   190.84.199.73   190.84.117.150      TCP   
 24            25      5.256138  190.84.117.150    190.84.199.73      TCP   
 25            26      5.257013   190.84.199.73   190.84.117.150      TCP   
 26            27      5.456759  190.84.117.150    190.84.199.73      TCP   
 36            37      6.300469   190.84.199.73   190.84.117.150      TCP   
 37            38      6.459867  190.84.117.150    190.84.199.73      TCP   
 46            47      8.709928   190.84.199.73   190.84.117.150      TCP   
 47            48      8.867577  190.84.117.150    190.84.199.73      TCP   

In [9]:
tcp["srcPort"], tcp["Info"] = tcp["Info"].str.split(" >  ", 1).str

/Users/aksaw/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
tcp["destPort"], tcp["Info"] = tcp["Info"].str.split(" ", 1).str

/Users/aksaw/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
tcp.srcPort=tcp.srcPort.str.rstrip()
tcp=tcp[tcp.srcPort.str.isnumeric()]

/Users/aksaw/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [12]:
# tcp=tcp.drop(["Info", "Protocol", "Time", "No."], axis=1)
tcp=tcp.drop(["Info", "Time", "No."], axis=1)

In [13]:
grouped=tcp.groupby(["Source", "srcPort", "Destination", "destPort", "Protocol"])
tcp['count'] = grouped["Length"].transform(len)
moreThanThree=tcp[tcp["count"]>=5].drop("count", axis=1)
moreThanThree=moreThanThree.sort_values(["Source", "srcPort", "Destination", "destPort", "Protocol"])

In [14]:
grouped=moreThanThree.groupby(["Source", "srcPort", "Destination", "destPort", "Protocol"])
firstThree=grouped.head(5)
assert len(firstThree)/5.0 == len(grouped.sum())
firstThree=firstThree.reset_index()
#first = firstThree
first = firstThree.Length[2::5,].reset_index(drop=True)
second = firstThree.Length[3::5,].reset_index(drop=True)
third = firstThree.Length[4::5,].reset_index(drop=True)
result = grouped.sum().reset_index()
result["firstSize"]=first
result["secondSize"]=second
result["thirdSize"]=third
result["flowSize"]=result["Length"]
result.drop("Length", axis=1, inplace=True)
result

,Source,srcPort,Destination,destPort,Protocol,firstSize,secondSize,thirdSize,flowSize
0,128.17.123.157,80,190.84.116.78,3451,TCP,1434,1434,1434,32130
1,128.17.123.157,80,190.84.116.78,3452,TCP,1434,1434,1434,222722
2,128.17.123.157,80,190.84.116.78,3457,TCP,790,694,1434,52550
3,128.17.123.157,80,190.84.116.78,3458,TCP,402,1363,697,74316
4,128.17.123.157,80,190.84.116.78,3505,TCP,194,193,194,39677
5,128.17.123.157,80,190.84.116.78,3507,TCP,194,194,194,103536
6,128.17.123.157,80,190.84.116.78,3510,TCP,195,193,194,10023
7,128.17.123.157,80,190.84.116.78,3512,TCP,195,195,195,1036
8,128.84.185.150,1733,190.84.116.154,1722,TCP,60,60,60,430
9,128.84.236.31,80,190.84.117.174,1129,TCP,1514,1514,1514,7945


In [15]:
result.to_csv(savename, index=False)

In [17]:
df = pd.read_csv(savename)
df

,Source,srcPort,Destination,destPort,Protocol,firstSize,secondSize,thirdSize,flowSize
0,128.17.123.157,80,190.84.116.78,3451,TCP,1434,1434,1434,32130
1,128.17.123.157,80,190.84.116.78,3452,TCP,1434,1434,1434,222722
2,128.17.123.157,80,190.84.116.78,3457,TCP,790,694,1434,52550
3,128.17.123.157,80,190.84.116.78,3458,TCP,402,1363,697,74316
4,128.17.123.157,80,190.84.116.78,3505,TCP,194,193,194,39677
5,128.17.123.157,80,190.84.116.78,3507,TCP,194,194,194,103536
6,128.17.123.157,80,190.84.116.78,3510,TCP,195,193,194,10023
7,128.17.123.157,80,190.84.116.78,3512,TCP,195,195,195,1036
8,128.84.185.150,1733,190.84.116.154,1722,TCP,60,60,60,430
9,128.84.236.31,80,190.84.117.174,1129,TCP,1514,1514,1514,7945


In [85]:
sum(df["flowSize"]>100000)

430

In [87]:
df.Protocol.unique()

array(['TCP', 'UDP', 'MANOLITO', 'H.225.0'], dtype=object)